In [1]:
import tempfile
import numpy as np
import tensorflow_datasets as tfds

import tensorflow as tf

from tensorflow.experimental import dtensor

print('TensorFlow version:', tf.__version__)

2025-11-27 14:19:23.404118: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-27 14:19:23.404191: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-27 14:19:23.487421: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-27 14:19:23.655392: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/xy/Desktop/ml/TF/legacy/lib/python3.10/site-pac

TensorFlow version: 2.15.1


In [2]:
def configure_virtual_cpus(ncpu):
  phy_devices = tf.config.list_physical_devices('CPU')
  tf.config.set_logical_device_configuration(phy_devices[0], [
        tf.config.LogicalDeviceConfiguration(),
    ] * ncpu)

configure_virtual_cpus(8)
DEVICES = [f'CPU:{i}' for i in range(8)]

tf.config.list_logical_devices('CPU')

2025-11-27 14:19:34.402112: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-27 14:19:34.736299: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-27 14:19:34.738337: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:CPU:1', device_type='CPU'),
 LogicalDevice(name='/device:CPU:2', device_type='CPU'),
 LogicalDevice(name='/device:CPU:3', device_type='CPU'),
 LogicalDevice(name='/device:CPU:4', device_type='CPU'),
 LogicalDevice(name='/device:CPU:5', device_type='CPU'),
 LogicalDevice(name='/device:CPU:6', device_type='CPU'),
 LogicalDevice(name='/device:CPU:7', device_type='CPU')]

In [3]:
train_data = tfds.load('imdb_reviews', split='train', shuffle_files=True, batch_size=64)
train_data

/home/xy/Desktop/ml/TF/legacy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dl Completed...: 0 url [00:00, ? url/s]
Generating splits...:   0%|                          | 0/3 [00:00<?, ? splits/s]
Generating train examples...: 0 examples [00:00, ? examples/s]
Generating train examples...: 1 examples [00:01,  1.17s/ examples]
Generating train examples...: 11514 examples [00:02, 6333.67 examples/s]
Generating train examples...: 23062 examples [00:03, 8628.12 examples/s]
                                                                        
Shuffling /home/xy/tensorflow_datasets/imdb_reviews/plain_text/incomplete.FW9490
Generating splits...:  33%|██████            | 1/3 [00:05<00:10,  5.27s/ splits]
Generating test examples...: 0 examples [00:00, ? examples/s]
Generating test examples...: 8490 examples [00:01, 8489.57 examples/s]
Generating test examples...: 20588 examples [00:02, 10611.88 examples/s]
                                                                        
Shuffling /home/xy/tensorflow_datasets/imdb_reviews/plain_text/incomplete.FW9490
Generati

Dataset imdb_reviews downloaded and prepared to /home/xy/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


<_PrefetchDataset element_spec={'label': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'text': TensorSpec(shape=(None,), dtype=tf.string, name=None)}>

In [4]:
text_vectorization = tf.keras.layers.TextVectorization(output_mode='tf_idf', max_tokens=1200, output_sequence_length=None)
text_vectorization.adapt(data=train_data.map(lambda x: x['text']))

In [5]:
def vectorize(features):
  return text_vectorization(features['text']), features['label']

train_data_vec = train_data.map(vectorize)
train_data_vec

<_MapDataset element_spec=(TensorSpec(shape=(None, 1200), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [6]:
class Dense(tf.Module):

  def __init__(self, input_size, output_size,
               init_seed, weight_layout, activation=None):
    super().__init__()

    random_normal_initializer = tf.function(tf.random.stateless_normal)

    self.weight = dtensor.DVariable(
        dtensor.call_with_layout(
            random_normal_initializer, weight_layout,
            shape=[input_size, output_size],
            seed=init_seed
            ))
    if activation is None:
      activation = lambda x:x
    self.activation = activation

    # bias is sharded the same way as the last axis of weight.
    bias_layout = weight_layout.delete([0])

    self.bias = dtensor.DVariable(
        dtensor.call_with_layout(tf.zeros, bias_layout, [output_size]))

  def __call__(self, x):
    y = tf.matmul(x, self.weight) + self.bias
    y = self.activation(y)

    return y

In [7]:
class BatchNorm(tf.Module):

  def __init__(self):
    super().__init__()

  def __call__(self, x, training=True):
    if not training:
      # This branch is not used in the Tutorial.
      pass
    mean, variance = tf.nn.moments(x, axes=[0])
    return tf.nn.batch_normalization(x, mean, variance, 0.0, 1.0, 1e-5)

In [8]:
def make_keras_bn(bn_layout):
  return tf.keras.layers.BatchNormalization(gamma_layout=bn_layout,
                                            beta_layout=bn_layout,
                                            moving_mean_layout=bn_layout,
                                            moving_variance_layout=bn_layout,
                                            fused=False)

In [9]:
from typing import Tuple

class MLP(tf.Module):

  def __init__(self, dense_layouts: Tuple[dtensor.Layout, dtensor.Layout]):
    super().__init__()

    self.dense1 = Dense(
        1200, 48, (1, 2), dense_layouts[0], activation=tf.nn.relu)
    self.bn = BatchNorm()
    self.dense2 = Dense(48, 2, (3, 4), dense_layouts[1])

  def __call__(self, x):
    y = x
    y = self.dense1(y)
    y = self.bn(y)
    y = self.dense2(y)
    return y

In [10]:
class MLPStricter(tf.Module):

  def __init__(self, mesh, input_mesh_dim, inner_mesh_dim1, output_mesh_dim):
    super().__init__()

    self.dense1 = Dense(
        1200, 48, (1, 2), dtensor.Layout([input_mesh_dim, inner_mesh_dim1], mesh),
        activation=tf.nn.relu)
    self.bn = BatchNorm()
    self.dense2 = Dense(48, 2, (3, 4), dtensor.Layout([inner_mesh_dim1, output_mesh_dim], mesh))


  def __call__(self, x):
    y = x
    y = self.dense1(y)
    y = self.bn(y)
    y = self.dense2(y)
    return y

In [11]:
WORLD = dtensor.create_mesh([("world", 8)], devices=DEVICES)

model = MLP([dtensor.Layout.replicated(WORLD, rank=2),
             dtensor.Layout.replicated(WORLD, rank=2)])

sample_x, sample_y = train_data_vec.take(1).get_single_element()
sample_x = dtensor.copy_to_mesh(sample_x, dtensor.Layout.replicated(WORLD, rank=2))
print(model(sample_x))

tf.Tensor([[-5.61041546 5.0473752]
 [-7.14075089 6.86515808]
 [-3.10483837 1.58168268]
 ...
 [6.87280321 -3.56776071]
 [8.27548695 -5.70918465]
 [-1.98807704 1.71495843]], layout="sharding_specs:unsharded,unsharded, mesh:|world=8|0,1,2,3,4,5,6,7|0,1,2,3,4,5,6,7|/job:localhost/replica:0/task:0/device:CPU:0,/job:localhost/replica:0/task:0/device:CPU:1,/job:localhost/replica:0/task:0/device:CPU:2,/job:localhost/replica:0/task:0/device:CPU:3,/job:localhost/replica:0/task:0/device:CPU:4,/job:localhost/replica:0/task:0/device:CPU:5,/job:localhost/replica:0/task:0/device:CPU:6,/job:localhost/replica:0/task:0/device:CPU:7", shape=(64, 2), dtype=float32)


2025-11-27 14:22:05.605454: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_stateless_random_normal_1147. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:22:05.620761: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_stateless_random_normal_1147. DTensor is (re-)computing its ExecutionFunctions.
2025-11-27 14:22:05.691114: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_stateless_random_normal_1168. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:22:05.691925: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_stateless_random_normal_1168. DTensor is (re-)computing its ExecutionFunctions.


In [12]:
def repack_local_tensor(x, layout):
  """Repacks a local Tensor-like to a DTensor with layout.

  This function assumes a single-client application.
  """
  x = tf.convert_to_tensor(x)
  sharded_dims = []

  # For every sharded dimension, use tf.split to split the along the dimension.
  # The result is a nested list of split-tensors in queue[0].
  queue = [x]
  for axis, dim in enumerate(layout.sharding_specs):
    if dim == dtensor.UNSHARDED:
      continue
    num_splits = layout.shape[axis]
    queue = tf.nest.map_structure(lambda x: tf.split(x, num_splits, axis=axis), queue)
    sharded_dims.append(dim)

  # Now we can build the list of component tensors by looking up the location in
  # the nested list of split-tensors created in queue[0].
  components = []
  for locations in layout.mesh.local_device_locations():
    t = queue[0]
    for dim in sharded_dims:
      split_index = locations[dim]  # Only valid on single-client mesh.
      t = t[split_index]
    components.append(t)

  return dtensor.pack(components, layout)

In [13]:
mesh = dtensor.create_mesh([("batch", 8)], devices=DEVICES)

model = MLP([dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh),
             dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh),])

2025-11-27 14:23:00.595520: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_stateless_random_normal_1221. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:23:00.596538: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_stateless_random_normal_1221. DTensor is (re-)computing its ExecutionFunctions.
2025-11-27 14:23:00.656824: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_stateless_random_normal_1242. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:23:00.657666: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_stateless_random_normal_1242. DTensor is (re-)computing its ExecutionFunctions.


In [14]:
def repack_batch(x, y, mesh):
  x = repack_local_tensor(x, layout=dtensor.Layout(['batch', dtensor.UNSHARDED], mesh))
  y = repack_local_tensor(y, layout=dtensor.Layout(['batch'], mesh))
  return x, y

sample_x, sample_y = train_data_vec.take(1).get_single_element()
sample_x, sample_y = repack_batch(sample_x, sample_y, mesh)

print('x', sample_x[:, 0])
print('y', sample_y)

x tf.Tensor({"CPU:0": [85.6979828 57.1319885 139.655975 ... 260.267944 438.011902 111.089973], "CPU:1": [117.437973 66.6539841 107.915977 ... 146.003967 260.267944 47.6099892], "CPU:2": [136.481964 215.831955 285.659943 ... 355.487915 206.309952 101.567978], "CPU:3": [107.915977 57.1319885 79.3499832 ... 63.4799881 203.135956 371.35791], "CPU:4": [206.309952 73.0019836 34.9139938 ... 82.5239792 44.4359894 69.8279877], "CPU:5": [95.2199783 219.005951 434.837891 ... 98.3939819 95.2199783 345.965912], "CPU:6": [174.569962 282.485931 38.0879898 ... 234.875946 79.3499832 79.3499832], "CPU:7": [215.831955 590.363892 107.915977 ... 238.049942 244.397949 82.5239792]}, layout="sharding_specs:batch, mesh:|batch=8|0,1,2,3,4,5,6,7|0,1,2,3,4,5,6,7|/job:localhost/replica:0/task:0/device:CPU:0,/job:localhost/replica:0/task:0/device:CPU:1,/job:localhost/replica:0/task:0/device:CPU:2,/job:localhost/replica:0/task:0/device:CPU:3,/job:localhost/replica:0/task:0/device:CPU:4,/job:localhost/replica:0/task:

In [15]:
# Refer to the CTL (custom training loop guide)
@tf.function
def train_step(model, x, y, learning_rate=tf.constant(1e-4)):
  with tf.GradientTape() as tape:
    logits = model(x)
    # tf.reduce_sum sums the batch sharded per-example loss to a replicated
    # global loss (scalar).
    loss = tf.reduce_sum(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=y))
  parameters = model.trainable_variables
  gradients = tape.gradient(loss, parameters)
  for parameter, parameter_gradient in zip(parameters, gradients):
    parameter.assign_sub(learning_rate * parameter_gradient)

  # Define some metrics
  accuracy = 1.0 - tf.reduce_sum(tf.cast(tf.argmax(logits, axis=-1, output_type=tf.int64) != y, tf.float32)) / x.shape[0]
  loss_per_sample = loss / len(x)
  return {'loss': loss_per_sample, 'accuracy': accuracy}

In [16]:
CHECKPOINT_DIR = tempfile.mkdtemp()

def start_checkpoint_manager(model):
  ckpt = tf.train.Checkpoint(root=model)
  manager = tf.train.CheckpointManager(ckpt, CHECKPOINT_DIR, max_to_keep=3)

  if manager.latest_checkpoint:
    print("Restoring a checkpoint")
    ckpt.restore(manager.latest_checkpoint).assert_consumed()
  else:
    print("New training")
  return manager

In [17]:
num_epochs = 2
manager = start_checkpoint_manager(model)

for epoch in range(num_epochs):
  step = 0
  pbar = tf.keras.utils.Progbar(target=int(train_data_vec.cardinality()), stateful_metrics=[])
  metrics = {'epoch': epoch}
  for x,y in train_data_vec:

    x, y = repack_batch(x, y, mesh)

    metrics.update(train_step(model, x, y, 1e-2))

    pbar.update(step, values=metrics.items(), finalize=False)
    step += 1
  manager.save()
  pbar.update(step, values=metrics.items(), finalize=True)

New training


2025-11-27 14:23:40.146566: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_train_step_1435. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:23:40.154183: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_train_step_1435. DTensor is (re-)computing its ExecutionFunctions.


390/391 [============================>.] - ETA: 0s - epoch: 0.0000e+00 - loss: 1.4115 - accuracy: 0.5329

2025-11-27 14:23:41.817364: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_train_step_13373. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:23:41.825002: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_train_step_13373. DTensor is (re-)computing its ExecutionFunctions.
2025-11-27 14:23:41.953759: I tensorflow/dtensor/cc/dtensor_device_util.cc:169] Broadcasting resource tensor to a dtensor resource tensor.
2025-11-27 14:23:41.953853: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_tf_function_save_13517. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:23:41.956333: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_tf_function_save_13517. DTensor is (re-)computing its ExecutionFunctions.


391/391 [==============================] - 1s 4ms/step - epoch: 1.0000 - loss: 0.9327 - accuracy: 0.5979


2025-11-27 14:23:43.429489: I tensorflow/dtensor/cc/dtensor_device_util.cc:169] Broadcasting resource tensor to a dtensor resource tensor.
2025-11-27 14:23:43.429579: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_tf_function_save_25495. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:23:43.432125: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_tf_function_save_25495. DTensor is (re-)computing its ExecutionFunctions.


In [18]:
mesh = dtensor.create_mesh([("batch", 4), ("model", 2)], devices=DEVICES)
model = MLP([dtensor.Layout([dtensor.UNSHARDED, "model"], mesh), 
             dtensor.Layout(["model", dtensor.UNSHARDED], mesh)])

2025-11-27 14:23:57.832163: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_stateless_random_normal_25519. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:23:57.833289: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_stateless_random_normal_25519. DTensor is (re-)computing its ExecutionFunctions.


2025-11-27 14:23:57.876325: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_stateless_random_normal_25540. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:23:57.877232: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_stateless_random_normal_25540. DTensor is (re-)computing its ExecutionFunctions.


In [19]:
def repack_batch(x, y, mesh):
  x = repack_local_tensor(x, layout=dtensor.Layout(['batch', dtensor.UNSHARDED], mesh))
  y = repack_local_tensor(y, layout=dtensor.Layout(['batch'], mesh))
  return x, y

In [20]:
num_epochs = 2
manager = start_checkpoint_manager(model)

for epoch in range(num_epochs):
  step = 0
  pbar = tf.keras.utils.Progbar(target=int(train_data_vec.cardinality()))
  metrics = {'epoch': epoch}
  for x,y in train_data_vec:
    x, y = repack_batch(x, y, mesh)
    metrics.update(train_step(model, x, y, 1e-2))
    pbar.update(step, values=metrics.items(), finalize=False)
    step += 1
  manager.save()
  pbar.update(step, values=metrics.items(), finalize=True)

Restoring a checkpoint


2025-11-27 14:24:18.972623: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_tf_function_restore_25651. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:24:18.974326: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_tf_function_restore_25651. DTensor is (re-)computing its ExecutionFunctions.
2025-11-27 14:24:19.101968: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_train_step_25806. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:24:19.109670: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_train_step_25806. DTensor is (re-)computing its ExecutionFunctions.


  0/391 [..............................] - ETA: 0s - epoch: 1.0000 - loss: 0.8866 - accuracy: 0.4844

2025-11-27 14:24:20.472140: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_train_step_34612. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:24:20.479632: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_train_step_34612. DTensor is (re-)computing its ExecutionFunctions.
2025-11-27 14:24:20.585344: I tensorflow/dtensor/cc/dtensor_device_util.cc:169] Broadcasting resource tensor to a dtensor resource tensor.
2025-11-27 14:24:20.585437: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_tf_function_save_34753. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:24:20.588074: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_tf_function_save_34753. DTensor is (re-)computing its ExecutionFunctions.
error: error: ''tfg.Casetfg.Case' op ' op branch #0branch #0 function argument 

391/391 [==============================] - 1s 3ms/step - epoch: 1.0000 - loss: 0.7261 - accuracy: 0.6984


2025-11-27 14:24:21.943959: I tensorflow/dtensor/cc/dtensor_device_util.cc:169] Broadcasting resource tensor to a dtensor resource tensor.
2025-11-27 14:24:21.944076: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_tf_function_save_43595. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:24:21.946684: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_tf_function_save_43595. DTensor is (re-)computing its ExecutionFunctions.
error: 'tfg.Case' op branch #0 function argument #3 type 'tensor<1200x24xf32>' is not compatible with corresponding operand type: 'tensor<1200x48xf32>'
2025-11-27 14:24:21.978135: W tensorflow/core/common_runtime/optimize_function_graph_utils.cc:615] Ignoring multi-device function optimization failure: INVALID_ARGUMENT: MLIR Graph Optimizer failed: 
error: 'tfg.Case' op branch #0 function argument #3 type 'tensor<1200x24xf32>' is not compatible with correspond

In [21]:
mesh = dtensor.create_mesh([("batch", 2), ("feature", 2), ("model", 2)], devices=DEVICES)
model = MLP([dtensor.Layout(["feature", "model"], mesh), 
             dtensor.Layout(["model", dtensor.UNSHARDED], mesh)])

2025-11-27 14:24:31.764215: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_stateless_random_normal_43619. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:24:31.765545: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_stateless_random_normal_43619. DTensor is (re-)computing its ExecutionFunctions.
2025-11-27 14:24:31.807715: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_stateless_random_normal_43640. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:24:31.808633: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_stateless_random_normal_43640. DTensor is (re-)computing its ExecutionFunctions.


In [22]:
def repack_batch_for_spt(x, y, mesh):
    # Shard data on feature dimension, too
    x = repack_local_tensor(x, layout=dtensor.Layout(["batch", 'feature'], mesh))
    y = repack_local_tensor(y, layout=dtensor.Layout(["batch"], mesh))
    return x, y

In [23]:
num_epochs = 2

manager = start_checkpoint_manager(model)
for epoch in range(num_epochs):
  step = 0
  metrics = {'epoch': epoch}
  pbar = tf.keras.utils.Progbar(target=int(train_data_vec.cardinality()))

  for x, y in train_data_vec:
    x, y = repack_batch_for_spt(x, y, mesh)
    metrics.update(train_step(model, x, y, 1e-2))

    pbar.update(step, values=metrics.items(), finalize=False)
    step += 1
  manager.save()
  pbar.update(step, values=metrics.items(), finalize=True)

Restoring a checkpoint


2025-11-27 14:24:47.677863: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_tf_function_restore_43751. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:24:47.680220: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_tf_function_restore_43751. DTensor is (re-)computing its ExecutionFunctions.
2025-11-27 14:24:47.791421: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_train_step_43908. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:24:47.799093: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_train_step_43908. DTensor is (re-)computing its ExecutionFunctions.
E0000 00:00:1764249887.849870   10019 op_level_cost_estimator.cc:1121] Incompatible Matrix dimensions
E0000 00:00:1764249887.850385   10033 op_level_cost_estimator.cc:1121] Incompatible Matrix dimensions
E0000 00

390/391 [============================>.] - ETA: 0s - epoch: 0.0000e+00 - loss: 0.6452 - accuracy: 0.7302WARNING:tensorflow:5 out of the last 5 calls to <function MultiDeviceSaver.save.<locals>.tf_function_save at 0x7b216e49dab0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


2025-11-27 14:24:49.184664: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_train_step_53498. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:24:49.192171: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_train_step_53498. DTensor is (re-)computing its ExecutionFunctions.
E0000 00:00:1764249889.242862   10025 op_level_cost_estimator.cc:1121] Incompatible Matrix dimensions
E0000 00:00:1764249889.243368   10021 op_level_cost_estimator.cc:1121] Incompatible Matrix dimensions
E0000 00:00:1764249889.245554   10027 op_level_cost_estimator.cc:1121] Incompatible Matrix dimensions
E0000 00:00:1764249889.245658   10031 op_level_cost_estimator.cc:1121] Incompatible Matrix dimensions
E0000 00:00:1764249889.246386   10033 op_level_cost_estimator.cc:1121] Incompatible Matrix dimensions
E0000 00:00:1764249889.246626   10023 op_level_cost_estimator.cc:1121] Incompatible Matrix dimensions
E00

381/391 [============================>.] - ETA: 0s - epoch: 1.0000 - loss: 0.5761 - accuracy: 0.7562WARNING:tensorflow:6 out of the last 6 calls to <function MultiDeviceSaver.save.<locals>.tf_function_save at 0x7b216e49d870> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


2025-11-27 14:24:50.679579: I tensorflow/dtensor/cc/dtensor_device_util.cc:169] Broadcasting resource tensor to a dtensor resource tensor.
2025-11-27 14:24:50.679701: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_tf_function_save_63263. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:24:50.682224: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_tf_function_save_63263. DTensor is (re-)computing its ExecutionFunctions.
error: 'tfg.Case' op branch #0 function argument #3 type 'tensor<600x24xf32>' is not compatible with corresponding operand type: 'tensor<1200x48xf32>'
2025-11-27 14:24:50.714768: W tensorflow/core/common_runtime/optimize_function_graph_utils.cc:615] Ignoring multi-device function optimization failure: INVALID_ARGUMENT: MLIR Graph Optimizer failed: 
error: 'tfg.Case' op branch #0 function argument #3 type 'tensor<600x24xf32>' is not compatible with correspondin

391/391 [==============================] - 1s 4ms/step - epoch: 1.0000 - loss: 0.5742 - accuracy: 0.7568


In [24]:
mesh = dtensor.create_mesh([("world", 1)], devices=DEVICES[:1])
mlp = MLP([dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh), 
           dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh)])

manager = start_checkpoint_manager(mlp)

model_for_saving = tf.keras.Sequential([
  text_vectorization,
  mlp
])

@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def run(inputs):
  return {'result': model_for_saving(inputs)}

tf.saved_model.save(
    model_for_saving, "/tmp/saved_model",
    signatures=run)

2025-11-27 14:25:02.271493: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_stateless_random_normal_63287. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:25:02.272662: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_stateless_random_normal_63287. DTensor is (re-)computing its ExecutionFunctions.
2025-11-27 14:25:02.296025: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_stateless_random_normal_63308. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:25:02.297016: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_stateless_random_normal_63308. DTensor is (re-)computing its ExecutionFunctions.
2025-11-27 14:25:02.335435: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_tf_function_restore_63363. DTensor is (re-)computing i

Restoring a checkpoint
INFO:tensorflow:Assets written to: /tmp/saved_model/assets


2025-11-27 14:25:02.879077: I tensorflow/dtensor/cc/dtensor_device_util.cc:169] Broadcasting resource tensor to a dtensor resource tensor.
2025-11-27 14:25:02.879101: I tensorflow/dtensor/cc/dtensor_device_util.cc:169] Broadcasting resource tensor to a dtensor resource tensor.
2025-11-27 14:25:02.879196: I tensorflow/dtensor/cc/dtensor_device.cc:1626] DTensor cache key lookup missed for __inference_tf_function_save_64384. DTensor is (re-)computing its SPMD transformation.
2025-11-27 14:25:02.882262: I tensorflow/dtensor/cc/dtensor_device.cc:1699] DTensor cache key lookup missed for __inference_tf_function_save_64384. DTensor is (re-)computing its ExecutionFunctions.
INFO:tensorflow:Assets written to: /tmp/saved_model/assets


In [25]:
sample_batch = train_data.take(1).get_single_element()
sample_batch

{'label': <tf.Tensor: shape=(64,), dtype=int64, numpy=
 array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
        1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1])>,
 'text': <tf.Tensor: shape=(64,), dtype=string, numpy=
 array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Wa

In [26]:
loaded = tf.saved_model.load("/tmp/saved_model")

run_sig = loaded.signatures["serving_default"]
result = run_sig(sample_batch['text'])['result']

In [27]:
np.mean(tf.argmax(result, axis=-1) == sample_batch['label'])

0.703125